## *** THIS TUTORIAL IS DEPRECATED ***

### Note that this Colab notebook is now deprecated.  We recommend using Super Annotate's notebook for Deeplabv3+ training instead:
https://github.com/superannotateai/model-deployment-tutorials/blob/main/OAK/SuperAnnotate_OAK_Deeplabv3%2B_Deployment.ipynb


In [1]:
!git clone https://github.com/GotG/models.git

Cloning into 'models'...
remote: Enumerating objects: 46531, done.
^C


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
%cd models/research/

/content/models/research


In [ ]:
!pip install tf_slim

     |████████████████████████████████| 358kB 5.2MB/s 


In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
# !python3 -c "import sys; print('\n'.join(sys.path))"

#The script below does the following:
* downloads Pascal 2012 dataset 
-converts it to tfrecord
-creates a new directory pascal_voc_seg under /content/models/research/deeplab/datasets
-downloads pretrained weights for deeplabv3_mnv2_pascal_train_aug from https://github.com/tensorflow/models/blob/master/research/deeplab/g3doc/model_zoo.md
-trains for 10 iterations (mock training)
-evaluates the model on the validation images
-runs inference on all the validation images and saves the inference targets amd results as files in /content/models/research/deeplab/datasets/pascal_voc_seg/exp/train_on_trainval_set_mobilenetv2/vis/segmentation_results 
-exports (freezes) the model: /content/models/research/deeplab/datasets/pascal_voc_seg/exp/train_on_trainval_set_mobilenetv2/export




In [ ]:
!bash deeplab/local_test_mobilenetv2.sh


Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] DeeplabModelTest.testBuildDeepLabWithDensePredictionCell
Instructions for updating:
Use `self.session()` or `self.cached_session()` instead.
W1027 13:31:41.498171 140550708238208 deprecation.py:323] From /usr/lib/python3.6/contextlib.py:60: TensorFlowTestCase.test_session (from tensorflow.python.framework.test_util) is deprecated and will be removed in a future version.
Instructions for updating:
Use `self.session()` or `self.cached_session()` instead.
2020-10-27 13:31:41.499468: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-10-27 13:31:41.528683: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-27 13:31:41.529348: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 

# In order to train on own data, follow the PascalVOC2012 annotation and directory structure format for own dataset. Note: Only SegmentationClass type annotations needed.SegmentationObject type not needed for semantic segmentation.
* make the necessary changes: /content/models/research/deeplab/datasets/download_and_convert_voc2012.sh
  * Download the images.

    BASE_URL="https://data.deepai.org/"

    FILENAME="PascalVOC2012.zip"
* the directory /content/models/research/deeplab/datasets/pascal_voc_seg/VOC2012/ImageSets/Segmentation contains 3 text files with filenames (no extension) of the images to be used for training, validation, and both combined if it is desired to train on the entire dataset (if not much data available for train/val split)
* Pascal uses a 50/50 split. 80/20 should be used in general for own dataset.


# For training,make changes accordingly in the local_test_mobilentv2.sh.
* train.py:
  * --training_number_of_steps="${NUM_ITERATIONS}" use a higher number of iterations (default is 10), e.g. 10,000
  * --train_split="trainval" - which images to train on (see above)
  * --train_crop_size="513,513" can be changed to "257,257" e.g.
    * train crop size should be a multiple of --output_stride + 1: 8 x 32 + = 257. for vis and eval, crop size should be larger than the larger image in the validation set
  * --train_batch_size=12 can be higher than 12, 16 or even more if it fits on the GPU
  * --fine_tune_batch_norm=true : this can be changed to false for faster training (less peformance, depending on dataset)
* --num_classes=21 - valid for Pascal (1 background). change according to own dataset 
* --export.py: choose the crop size you wish to use as input for the model

# Install OpenVINO

In [ ]:
%%time
%%capture
## install tools. Open Vino takes some time to download: 10-15 min sometimes.
!sudo apt-get install -y pciutils cpio
!sudo apt autoremove
## downnload installation files
!wget http://registrationcenter-download.intel.com/akdlm/irc_nas/16345/l_openvino_toolkit_p_2020.1.023.tgz
# !wget http://registrationcenter-download.intel.com/akdlm/irc_nas/16612/l_openvino_toolkit_p_2020.2.120.tgz
# !wget http://registrationcenter-download.intel.com/akdlm/irc_nas/16803/l_openvino_toolkit_p_2020.4.287.tgz
path = "l_openvino_toolkit_p_2020.1.023.tgz"
# path = "l_openvino_toolkit_p_2020.2.120.tgz"
# path = "l_openvino_toolkit_p_2020.4.287.tgz"
# path = "/content/software/Intel OpenVINO 2019 R3.1/l_openvino_toolkit_p_2019.3.376.tgz"
## install openvino
!tar xf "{path}"
%cd l_openvino_toolkit_p_2020.1.023/
# %cd l_openvino_toolkit_p_2020.4.287/
!./install_openvino_dependencies.sh && \
    sed -i 's/decline/accept/g' silent.cfg && \
    ./install.sh --silent silent.cfg

CPU times: user 242 ms, sys: 82.1 ms, total: 324 ms
Wall time: 1min 30s


## Convert the frozen TF model to OpenVINO Intermediate Representation: .xml and .bin. as input shape choose the crop size used above in export.py , e.g. 256

In [ ]:
!source /opt/intel/openvino/bin/setupvars.sh && \
    python /opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model  /content/models/research/deeplab/datasets/pascal_voc_seg/exp/train_on_trainval_set_mobilenetv2/export/frozen_inference_graph256.pb \
--model_name deeplab_v3_plus_mnv2_decoder_256.pb \
--input_shape [1,256,256,3] \
--data_type FP16 \
--reverse_input_channel \
--mean_values "[127.5, 127.5, 127.5]" \
--scale_values "[127.5, 127.5, 127.5]" \
--output_dir /content/

[setupvars.sh] OpenVINO environment initialized
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/content/models/research/deeplab/datasets/pascal_voc_seg/exp/train_on_trainval_set_mobilenetv2/export/frozen_inference_graph256.pb
	- Path for generated IR: 	/content/
	- IR output name: 	deeplab_v3_plus_mnv2_decoder_256.pb
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,256,256,3]
	- Mean values: 	[127.5, 127.5, 127.5]
	- Scale values: 	[127.5, 127.5, 127.5]
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	True
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared l